We use three base classifiers: XGBClassifier, LGBMClassifier, and GradientBoostingClassifier. With the preprocessed training data, we use NeverGrad to find the optimal hyperparameters for all three classifiers. We create a VotingClassifier using the three classifiers that are initialised with these optimal hyperparameters and uses soft or weighted voting. 

We train the ensemble model on the y_train and preprocessed x_train data. Finally, we make predictions on the preprocessed x_test data, and output the predictions in a csv file.  

In [3]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as ltb
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier


x_train = pd.read_csv('x_train_preprocessed.csv', header=0, index_col=['id'])
y_train = pd.read_csv('y_train.csv', index_col=['id'])
x_test = pd.read_csv('x_test_preprocessed.csv', header=0, index_col=['id'])
xgb_clf = XGBClassifier(learning_rate=0.08553505693555381, max_depth=8, min_child_weight=7.361897515910249,
                                                        subsample=0.46469607072105973, colsample_bytree=0.3613230580519376, gamma= 0.016109487457977058, random_state=42)
lgbm_clf = ltb.LGBMClassifier(learning_rate=0.178780869915817, max_depth=31,
                                                        min_child_samples=64, n_estimators=342, num_leaves=225)
gb_clf = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=7, 
                                                                                    min_samples_split=60, min_samples_leaf=9, subsample=1,
                                                                                    max_features=50, random_state=0)

# Create a voting classifier
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('lgbm', lgbm_clf), ('gb', gb_clf)],
    voting='soft',  # 'hard' for majority voting, 'soft' for weighted voting
    weights=[3, 1, 4]
)

# Train the ensemble model
voting_clf.fit(x_train, y_train)

# Make predictions on the test set
predictions = voting_clf.predict(x_test)

submission = pd.DataFrame({"id": np.arange(len(predictions)), "y": predictions})
submission.to_csv(f"submission33.csv", index=False)




c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#pip install xgboost nevergrad


In [4]:
pip install -U colorama


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nevergrad as ng
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import pandas as pd


# Load dataset
X = pd.read_csv('x_train_preprocessed.csv', header=0, index_col=['id'])
y = pd.read_csv('y_train.csv', index_col=['id'])

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=42)

# Define the objective function for optimization (minimization)
def objective_function(learning_rate, max_depth, min_child_weight, subsample, colsample_bytree, gamma):
    # Round integer hyperparameters
    max_depth = int(max_depth)
    min_child_weight = int(min_child_weight)
    
    # Create an XGBoost model with the specified hyperparameters
    model = XGBClassifier(
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        random_state=42
    )

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = model.predict(X_valid)

    # Calculate accuracy (you can use other metrics as needed)
    accuracy = accuracy_score(y_valid, y_pred)

    # Nevergrad minimizes the objective function, so return the negative accuracy
    return -accuracy

# Define the search space for hyperparameters
parametrization = ng.p.Instrumentation(
    learning_rate=ng.p.Log(lower=1e-4, upper=1),
    max_depth=ng.p.Scalar(lower=3, upper=10),
    min_child_weight=ng.p.Scalar(lower=1, upper=10),
    subsample=ng.p.Log(lower=0.1, upper=1),
    colsample_bytree=ng.p.Log(lower=0.1, upper=1),
    gamma=ng.p.Log(lower=1e-4, upper=1)
)

# Choose the optimizer
optimizer = ng.optimizers.NGO(parametrization=parametrization, budget=500)

# Perform optimization
bestX = optimizer.minimize(objective_function, batch_mode=False)

# Get the best hyperparameters

# Train the final model with the best hyperparameters
with open("best_hyperparameters500.txt", "w") as file:

        file.write(f"{bestX.value}")
# Make predictions on the test set or use the model as needed


c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\cma\evolution_strategy.py:2981: InjectionWarning: orphanated injected solution {'iteration': 1, 'index': 0, 'counter': 0}
                        This could be a bug in the calling order/logics or due to
                        a too small popsize used in `ask()` or when only using
                        `ask(1)` repeatedly. Please check carefully.
                        In case this is desired, the warning can be surpressed with
                        ``warnings.simplefilter("ignore", cma.evolution_strategy.InjectionWarning)``
                        
  warnings.warn("""orphanated injected solution %s
c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\cma\evolution_strategy.py:2981: InjectionWarning: orphanated injected solution {'iteration': 2, 'index': 0, 'counter': 1}
                        This could be a bug in the calling order/logics or due to
                        a too small pop

<class 'tuple'>


AttributeError: 'tuple' object has no attribute 'items'